# TP 2

#### Grupo 4: Aschieri Juan Pablo, Civini Diego Emanuel, Rivero Joaquin, Rovira Rossel Francisco

In [83]:
from math import e, log
import time
from heapq import heappush, heappop
from scipy.stats import poisson
import matplotlib.pyplot as plt
import itertools
import simpy
import numpy as np
import random
from statistics import mean

## Generador de numeros aleatorio del TP1

In [77]:
class Xorshift_generator:
    def __init__(self, seed):
        self.x = seed
        self.y = 463820370
        self.z = 138409373
        self.w = 12872349

    def rand(self):
        tmp = ( self.x ^ (self.x << 13) ) 
        self.x = self.y
        self.y = self.z
        self.z = self.w
        self.w = (self.w^(self.w>>17) ) ^ (tmp^(tmp>>5))
        return self.w & 0xffffffff
    
    def get_n_rand(self, n):
        rand_numbers = []
        for i in range(n):
            rand_numbers.append(self.rand())
        return rand_numbers

In [78]:
class Uniform_generator:
    def __init__(self, seed, scale= 1, displacement= 0):
        self.xorshift_genertor = Xorshift_generator(seed)
        self.scale = scale
        self.displacement = displacement

    def rand(self):
        n = self.xorshift_genertor.rand() / ((2**32)-1)
        return n * self.scale + self.displacement
    
    def get_n_rand(self, n):
        rand_numbers = []
        for i in range(n):
            rand_numbers.append(self.rand())
        return rand_numbers

In [79]:
class Exp_generator:
    def __init__(self, seed, parameter=1):
        self.parameter = parameter
        self.Uniform_generator = Uniform_generator(seed)

    def rand(self):
        r = self.Uniform_generator.rand()
        if self.parameter >=1:
            return log(r/self.parameter) / (-1*self.parameter)
        else: 
            return -log(1 - r) / self.parameter

    def get_n_rand(self, n):
        rand_numbers = []
        for i in range(n):
            rand_numbers.append(self.rand())
        return rand_numbers

descubri que los primeros dos numeros del generador son muy distintos y despues se acomoda. lo de abajo hay que borrarlo  
tambien sospecho que por lo general da media menor que 6 mas seguido que mayor que 6, creo que tienee sentido pero mmm

In [ ]:
seed = int(time.time())

exp_generator = Exp_generator(seed, 1/6)

b = exp_generator.rand()
b = exp_generator.rand()
a = []

for i in range(100):
    n = exp_generator.rand()
    a.append(n)
    if n < 0:
        print(n)

print(a)
print(sum(a)/len(a))

## Ejercicio 1

Definimos un generador de una distribucion de poisson porque lo utilizaremos mas adelante.

In [ ]:
def factorial(n):
    resultado = 1
    for i in range(1, n + 1):
        resultado *= i
    return resultado

def poisson_probability(parameter, i, previous_probability):
    if previous_probability == 0:
        return poisson.pmf(i, parameter)
    return (parameter/i)*previous_probability

class Poisson_generator:
    def __init__(self, seed, parameter):
        self.parameter = parameter
        self.Uniform_generator = Uniform_generator(seed)

    def rand(self):
        r = self.Uniform_generator.rand()

        accumulated_probability = poisson_probability(self.parameter, 0, 0)
        previous_probability = accumulated_probability
        i = 0

        while(accumulated_probability < r):
            i += 1
            previous_probability = poisson_probability(self.parameter, i, previous_probability)
            accumulated_probability += previous_probability

        return i
    
    def get_n_rand(self, n):
        rand_numbers = []
        for i in range(n):
            rand_numbers.append(self.rand())
        return rand_numbers

Asumimos que el satélite da siempre la misma vuelta al mundo que tarda lo mismo, y que se puede pensar al tiempo en que tarda el satelite en llegar a una posicion a la que puede sacar una foto de la request como una uniforme entre 0 y el tiempo en que tarda en dar la vuelta al mundo, ya que los puntos sobre el globo se distribuyen uniformemente.

Para simular el problema lo que hacemos es definir un delta de tiempo (time step), que sera de cada que intervalo discreto avanzaremos el tiempo de la simulacion. Por cada iteracion generamos un valor de una distribucion de poisson que representa la cantidad de arribos de solicitudes que llegaron en el ultimo intervalo de tiempo. Por cada solicitud de tiempo que halla llegado luego generamos un valor de tiempo con una distribucion uniforme entre 0 y el satelite_perior (tiempo en dar la vuelta al mundo del satelite), que representa el tiempo que tardara el satelite en estar en posicion para poder sacar la foto. Por cada iteracion avanzamos el tiempo de todas las request en la cola, y agregamos las nuevas, ademas de ir anotando datos como cuantas veces hubo cada longitud de cola en todas las iteraciones, y cuanto fue el tiempo para servir a la primera request para cada longitud de cola.

In [ ]:
def simulate_satelite_request_queue_length(time_step, iterations, request_per_unit_time, satelite_period):
    generator_amount_of_request = Poisson_generator(int(time.time()) * 8647359734, request_per_unit_time*time_step)
    generator_time_to_serve = Uniform_generator(int(time.time()) * 345678976, satelite_period)
    simulated_queue = []
    observed_queue_length = {}
    time_to_serve_first_for_queue_length = {}

    for _ in range(iterations):
        advance_time(simulated_queue, time_step)
        get_acumulated_requests(generator_amount_of_request, simulated_queue, generator_time_to_serve)    
        observed_queue_length[len(simulated_queue)] = observed_queue_length.get(len(simulated_queue), 0) + 1
        if len(simulated_queue) != 0:
            time_to_serve_first_for_queue_length.setdefault(len(simulated_queue), []).append(simulated_queue[0])
            
    return get_frequencies(observed_queue_length, iterations), get_mean_times(time_to_serve_first_for_queue_length)

In [ ]:
def advance_time(simulated_queue, time_step):
    for i in range(len(simulated_queue)):
        simulated_queue[i] -= time_step

    while (len(simulated_queue) != 0 and simulated_queue[0] <= 0):
        item = heappop(simulated_queue)
        #print(f'Se elimino una request de tiempo: {item}')

def get_acumulated_requests(generator_amount_of_request, simulated_queue, generator_time_to_serve):
    amount_of_requests = generator_amount_of_request.rand()
    for request in range(amount_of_requests):
        #por ahora asumimos que el time to serve es uniforme, porque se hacen requests uniformemente alrededor del mundo
        heappush(simulated_queue, generator_time_to_serve.rand())

def get_frequencies(observed_queue_length, iterations):
    #print(f'Entro a get_frequencies con dicc = {observed_queue_length}')
    observed_frequencies = []
    #print(f'El maximo es: {max(observed_queue_length.keys())}')
    for j in range(max(observed_queue_length.keys()) + 1):
        times_observed = observed_queue_length.get(j, 0)
        #print(f'Para {j} se observo {times_observed}')
        observed_frequencies.append(times_observed/iterations)
    return observed_frequencies

def get_mean_times(time_to_serve_for_queue_length):
    mean_times = {}

    for key in time_to_serve_for_queue_length:
        sum_of_times = 0
        for j in time_to_serve_for_queue_length[key]:
            sum_of_times += j
        mean_times[key] = sum_of_times/len(time_to_serve_for_queue_length[key])
    
    return mean_times

Simulamos un ejemplo donde tomamos que el satelite tarde 48 horas en dar la vuelta al mundo, y que llegan 10 requests por dia en promedio. Usamos un intervalo de media hora y realizamos 50 000 horas de simulacion

In [ ]:
frequencies, times = simulate_satelite_request_queue_length(0.5, 100000, 5/12, 48)

Podemos ver que como en general se acumulan las request hasta llegar al rededor de 10 y luego tiende a disminuir la cantidad que hay

In [ ]:
x_axis = list(range(len(frequencies)))
y_axis = frequencies
plt.plot(x_axis, y_axis)
plt.show()

El paper llegaba a la conclusion de que cuantas mas solicitudes hayan en la cola, el tiempo que se tarda en servir estas request es menor. Para verificar esto miramos el siguiente grafico que muestra el tiempo promedio que tarda el satelite en servir al request mas cercano

In [ ]:
plt.bar(times.keys(), times.values())
plt.xlabel("Queue length")
plt.ylabel("Avarege time to serve first request")
plt.show()

Vemos que en este caso asi sucede, cuantas mas solicitudes hay en la cola menor es el tiempo para tomar la foto. Esto tiene sentido ya que cuantas mas request hayan en espera mas chances habra de que haya alguna mas cercana.
Si bien el grafico solo muestra la primera requet, como esto se modela como un proceso de poisson, por propiedad de perdidad de memoria se puede ver que se cumplira para el tiempo entre todos los arribos. 

## Ejercicio 2

In [80]:
BUFFER_SIZE = 100
SIM_TIME = 100

- Factor de ocupación del sistema. (listo)
- Cantidad media de solicitudes en el buffer (listo)
- Cantidad media de solicitudes en el sistema (listo)
- Tiempo medio que las solicitudes están en el buffer (listo)
- Tiempo medio de solicitudes en el sistema (listo)
- Determinar el tamaña del buffer para que la probabilidad de que haya lugar para recibir solicitudes sea
como mucho 3%.
- Evaluar la posibilidad de reemplazar las dos máquinas por una sola que procesa todos las solicitudes en
un tiempo fijo de 5 segundos. Si el costo de tener solicitudes en el buffer es 0,2 USD/minuto y el costo
de modificar la máquina es 2000 USD, Cuánto tiempo tomará amortizar el inversión?

De esta manera cada maquina es un proceso y desp hay otro proceso que se encarga de llenar las lista con requests para que cada maquina vaya desencolando 
y procesando requests

In [ ]:
class Machine:
    
    def __init__(self,id):
        self.working_time = 0
        self.requests_taken = 0
        self.id = id
        self.req_time_stats = []
    
    def process_request(self, request):
        self.requests_taken += 1
        self.working_time += request

    def add_req_time_stat(self, time_tup):
        # The tuple has the following format: (req_buffer_time, req_system_time, req_finish_time)
        self.req_time_stats.append(time_tup)


In [ ]:
"""
Con esta implementacion, las maquinas chequean que haya requests en la cola cada 1 seg. Osea, que si la cola estaa vacia, la maquina cada 1 seg va a 
chequear si llego un nuevo mensaje. Ver con los chicos si estan de acuerdo con esto (o tamb preguntar al de la practica). Porque otro caso que CREO que se puede
hacer es que apenas llegue un request, la maquina ya sepa y lo empieze a procesar all instante.
Tambien ver que onda el arrribo de mensajes, estan llegando cada mucho tiempo y los procesa re rapido entonces la cola siempre esta vacia
"""
def request_generator(env, requests_center, requests_buffer, seed):
    """Las solicitudes llegan siguiendo un proceso Poisson con una frecuencia media de 10
    por minuto. Por ende el tiempo entre 2 solicitudes consecutivas seguira una distribucion
    Exponencial de paremtro 10."""
    exponential_gen = Exp_generator(seed, 1/6) 
    
    print("Requests start to arrive")
    for i in itertools.count():
        time_for_next_request = exponential_gen.rand()
        #time_for_next_request = np.random.exponential(6) # El parametro en esta lib tiene que ser b=1/lambda

        print(f"The next request will arrive in {time_for_next_request}, we are at {env.now}")
        yield env.timeout(time_for_next_request)
        with requests_center.request() as req:
            yield req
            if len(requests_buffer.items) == BUFFER_SIZE:
                print(f"The buffer is full and cannot accept more requests. This requests has been dropped at time {env.now}")
            else:
                print(f"A request arrived and was buffered at time {env.now}")
                yield requests_buffer.put(env.now)
    print("Requests will stop to arrive")


In [ ]:
def processing_machine(env, requests_center, requests_buffer, seed, machine):
    exponential_gen = Exp_generator(seed, 1/10)
    print(f"The machine {machine.id} starts.")
    while True:
        
        time_to_process = 0.001
        request_processed = False
        #print(f"Machine {machine.id} will try to process a request at {env.now}")
        with requests_center.request() as req:
            yield req
            
            if len(requests_buffer.items) != 0:
                print(f"A request is starting to be processed by machine {machine.id} at {env.now}")
                arrival_time = yield requests_buffer.get()
                print(arrival_time)
                buffer_time = env.now - arrival_time
                print(f"La cantidad de requests en el buffer es {len(requests_buffer.items)}")
                time_to_process = exponential_gen.rand()
                #time_to_process = np.random.exponential(10)
                machine.process_request(time_to_process)
                print(f"Machine {machine.id} will finish at {time_to_process + env.now}")
                request_processed = True
                
        # If there are no requests to process in the buffer, the timeout is of 0 seconds.
        yield env.timeout(time_to_process)
        if request_processed:
            system_time = env.now - arrival_time
            finish_time = arrival_time + system_time
            time_tup = (buffer_time, system_time, finish_time)
            machine.add_req_time_stat(time_tup)
            print(f"Machine {machine.id} has finished processing the request at {env.now}")

In [ ]:
def buffer_stats(env, buffer, amount):
    while True:
        amount.append(len(buffer.items))
        yield env.timeout(1)

podemos usar nuestro generador, ronda cantidad de llegadas 11/12 en el primer minuto (empiricamente), varias veces se pasa y muy pocas veces de menos, el otro es mas uniforme, puede dar 8 o 13.  
voy a dejar el otro por ahora

In [ ]:
env = simpy.Environment()
init = env.now
requests_center = simpy.Resource(env, 1)
requests_buffer = simpy.Store(env, BUFFER_SIZE)
env.process(request_generator(env, requests_center, requests_buffer, int(time.time())))

amount = []
machine_1 = Machine(1)
machine_2 = Machine(2)
env.process(buffer_stats(env, requests_buffer, amount))
env.process(processing_machine(env, requests_center, requests_buffer, int(time.time()), machine_1))
#env.process(processing_machine(env, requests_center, requests_buffer, int(time.time()), machine_2))

env.run(until=SIM_TIME)

end = env.now

Ocupation factor con una maquina

In [ ]:
program_time = end - init
ocupation_factor = machine_1.working_time/SIM_TIME
print(f"ocupation_factor: {ocupation_factor}")
print(machine_1.working_time)

Ocupation factor con dos maquinas

In [ ]:
ocupation_factor = (machine_1.working_time + machine_2.working_time)/(program_time*2)
print(f"ocupation_factor: {ocupation_factor}")

Cantidad media de elementos en el buffer: a cada segundo mido la cantidad de elementos en el buffer, luego divido la suma por la cantidad total de mediciones

In [ ]:
cant = 0
for i in amount:
    cant += i

cant = cant/len(amount)
print(f"cantidad media de elementos en el buffer: {cant}")  

Sacamos la cantidad media de solicitudes en el sistema

In [ ]:
total_req_time_stats = machine_1.req_time_stats + machine_2.req_time_stats

req_quantity = []
for second in range(SIM_TIME):
    interval_req_quantity = 0
    for time_tup in total_req_time_stats:
        finish_time = time_tup[2]
        arrival_time = finish_time - time_tup[1] # finish_time - system_time
        if arrival_time < second < finish_time:
            """ 
            If the second we are checking is between the finish time and arrival time
            it means that the request was living on the system at that second.
            """
            interval_req_quantity += 1
    req_quantity.append(interval_req_quantity)
print(req_quantity)
print(mean(req_quantity))


    
    

## EJERCICIO 3

In [81]:
class Truck:

    def __init__(self, env, id, arrival_time):
        self.env = env
        self.id = id
        self.arrival_time = arrival_time
        self.download_shipment_time = None
        self.finish_time = None


    def download_shipment(self, distribution_center, type):
        exp_generator = Exp_generator(int(env.now), 1/5)
        if type == 'barcode':
            exp_generator = Exp_generator(int(env.now), 1/30)

        n1 = exp_generator.rand() # TO-DO: Ver porq nuestro generados los primeros numero son chiquitos
        n2 = exp_generator.rand()
        with distribution_center.request() as req:
            yield req
            self.download_shipment_time = env.now
            time_to_download = exp_generator.rand()
            print(f'The truck {self.id} started to download the shipment at {env.now}, it will finish at {env.now + time_to_download}')
            yield env.timeout(time_to_download)
        self.finish_time = env.now
        print(f'The truck {self.id} finished at {env.now}')
            

def truck_generator(env, distribution_center, seed, trucks, type):
    exp_generator = Exp_generator(seed, 1/12)
    for i in itertools.count():
        time_for_next_truck = exp_generator.rand()
        print(f'The next truck will arrive in {time_for_next_truck}')
        yield env.timeout(time_for_next_truck)
        print(f'The truck {i} arrived at {env.now}')
        truck = Truck(env, i, env.now)
        env.process(truck.download_shipment(distribution_center, type))
        trucks.append(truck)


Escenario en el que se usa codigo de barras para descargar y hay 4 darsenas

In [82]:

random.seed(22)
env = simpy.Environment()
distribution_center = simpy.Resource(env, 4)

trucks = []
env.process(truck_generator(env, distribution_center, int(time.time()) * 235462324, trucks, 'barcode'))

env.run(500)

The next truck will arrive in 1.1757733802229244
The truck 0 arrived at 1.1757733802229244
The next truck will arrive in 30.721401857562935
The truck 0 started to download the shipment at 1.1757733802229244, it will finish at 60.6026430556568
The truck 1 arrived at 31.89717523778586
The next truck will arrive in 12.560618941798008
The truck 1 started to download the shipment at 31.89717523778586, it will finish at 91.31302737728089
The truck 2 arrived at 44.45779417958387
The next truck will arrive in 0.09194971514311881
The truck 2 started to download the shipment at 44.45779417958387, it will finish at 103.86638015741738
The truck 3 arrived at 44.54974389472699
The next truck will arrive in 9.717763337248192
The truck 3 started to download the shipment at 44.54974389472699, it will finish at 103.95832987256048
The truck 4 arrived at 54.26750723197518
The next truck will arrive in 6.37561387805069
The truck 0 finished at 60.6026430556568
The truck 4 started to download the shipment at

Estadísticas:
-  Simular ambos escenarios utilizando Simpy o programación tradicional.
-  Cantidad media de camiones en el sistema (listo)
-  Cantidad media de camiones en cola (listo)
-  Tiempo medio de camiones en el sistema (listo)
-  Tiempo medio de camiones en cola (listo)
-  Qué recomendación se puede hacer sobre el cierre de la dársena propuesto

In [93]:
# We cheack for every second how many trucks were in the system, buffer, their times, etc.
system_quantity = []
buffer_quantity = []
trucks_time_stats = []
for second in range(500):
    trucks_in_system = 0
    trucks_in_buffer = 0
    for truck in trucks:
        if not all([truck.download_shipment_time,truck.finish_time]):
            # If the truck neever got out of the buffer or system beefore the simulation ended, their times cannot be taken into account
            continue

        if truck.arrival_time < second < truck.finish_time:
            trucks_in_system += 1

        if truck.arrival_time < second < truck.download_shipment_time:
            trucks_in_buffer += 1
    system_quantity.append(trucks_in_system)
    buffer_quantity.append(trucks_in_buffer)

trucks_system_time = [truck.finish_time - truck.arrival_time for truck in trucks if all([truck.download_shipment_time,truck.finish_time])]
trucks_buffer_time = [truck.download_shipment_time - truck.arrival_time for truck in trucks if all([truck.download_shipment_time,truck.finish_time])]

print(mean(trucks_system_time))
print(mean(trucks_buffer_time))

68.89417289891588
9.526618215840141


Escenario en el que se usa RFID para descargar y hay 3 darsenas

In [ ]:
random.seed(22)

env = simpy.Environment()
distribution_center = simpy.Resource(env, 3)

trucks = []
env.process(truck_generator(env, distribution_center, int(time.time()), trucks, 'rfid'))

env.run(500)